# Fix simulation sier results
7 genes' sier PIP are greater than 1 since the summation of whole genome varbvs PIPs are greater than 1.

In [1]:
import pandas as pd, numpy as np
import os
from collections import Counter
cwd = os.path.expanduser("/home/min/GIT/cnv-gene-mapping/output")
name = "simu_dele_30_0529"
prefix = "deletion.X.colnames_b30.simu_dele_30_0529"

In [2]:
index = pd.read_csv(f"{cwd}/{name}/{prefix}.X.block_index.csv", sep = "\t", header = None, names = ["start", "end"])

In [4]:
index.shape

(565, 2)

In [17]:
for i, item in index.iterrows():
    sumpip = np.sum(np.loadtxt(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.varbvs_pip", usecols = 1))
    if sumpip > 1:
        sier = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = None, names = ["gene", "pip"])
        if max(sier["pip"]) > 1:
            print (item, max(sier["pip"]))
            sier["pip"] = [x/sumpip for x in sier["pip"]]
            print (sier)
            sier.to_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = False, index = False)

start    95
end      96
Name: 19, dtype: int64 1.000000000000005
     gene           pip
0  NBPF20  4.225934e-23
1  NBPF19  9.958327e-01
